In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

## CHANGE IF EXPORT NEEDED:

In [ ]:
EXPORT = False # whether to export data and images

### 0. Define Plot Function

In [ ]:
def plot_fx(df, variable, label, ymax = None, EXPORT = False):
    plt.figure(figsize=(12, 4)) # create figure
    
    plt.plot(df.index, df[variable], label=label) # plot mean volatility
    
    plt.fill_between(df.index, 0, df[variable],
                     where=df["Regime"] == "High", color="red", alpha=0.2, label="High Regime") # fill area under high regime with red
    plt.fill_between(df.index, 0, df[variable],
                     where=df["Regime"] == "Low", color="green", alpha=0.2, label="Low Regime") # fill area under low regime with green

    if ymax: # limit outlier spikes for visibility
        plt.ylim(0, 0.015)
    
    plt.legend() # set legend
    plt.title(f"Cross-sectional {variable} Regimes") # set title
    plt.tight_layout() # ensure everything fits

    if EXPORT:
        plt.savefig(f"fx_{label}_regimes_chart.png") # save chart
    plt.show() # show plot

### 1. Define path, pairs, rolling window, and regime percentiles

In [ ]:
data_dir = Path("data/")
pairs = ['EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'AUDUSD', 'USDCAD', 'NZDUSD']

# volatility presets
vol_window = 21 # 21-day window, roughly 1 month of trading days
vol_lower_percentile = 0.3 # 30th percentile
vol_upper_percentile = 0.7 # 70th percentile

# dispersion presets
disp_window = 21 # 21-day window, roughly 1 month of trading days
disp_lower_percentile = 0.3 # 30th percentile
disp_upper_percentile = 0.7 # 70th percentile

### 2. Load Data

In [ ]:
dfs = {}

for pair in pairs:
    df = pd.read_csv(data_dir / f"{pair}.csv")
    df["Date"] = pd.to_datetime(df["Date"]) # ensure date format
    df.set_index("Date", inplace=True) # set date as index
    df = df[["Price"]].rename(columns={"Price": pair}) # rename price to pair
    df = df.drop_duplicates() # drop duplicates
    dfs[pair] = df # assign to dict

df_all = pd.concat(dfs.values(), axis=1).ffill() # concat and fill forward for misaligned days

### 3. Compute Volility + Regime

In [ ]:
# log returns, needed for both volatility and dispersion
rets = np.log(df_all / df_all.shift(1)).dropna() # log returns

In [ ]:
# volatility and mean across currency pairs
vol = rets.rolling(vol_window).std().dropna() # rolling standard deviation
vol_mean = vol.mean(axis=1) # mean across currency pairs

# expanding percentiles (causal)
vol_low = vol_mean.expanding().quantile(vol_lower_percentile)
vol_high = vol_mean.expanding().quantile(vol_upper_percentile)

# define regimes
vol_regime = pd.Series("Mid", index = vol_mean.index)
vol_regime[vol_mean < vol_low] = "Low"
vol_regime[vol_mean > vol_high] = "High"

# output dataframe
vol_df = pd.DataFrame({
    "Volatility": vol_mean,
    "Regime": vol_regime
})

vol_df = vol_df.iloc[vol_window:] # exclude initial lookback period 

### 4. Compute Dispersion + Regime

In [ ]:
# dispersion across currency pairs
disp = rets.std(axis=1)

# expanding percentiles (causal)
disp_low = disp.expanding().quantile(disp_lower_percentile)
disp_high = disp.expanding().quantile(disp_upper_percentile)

# define regimes
disp_regime = pd.Series("Mid", index = disp.index)
disp_regime[disp < disp_low] = "Low"
disp_regime[disp > disp_high] = "High"

# output dataframe
disp_df = pd.DataFrame({
    "Dispersion": disp,
    "Regime": disp_regime
})

disp_df = disp_df.iloc[disp_window:] # exclude initial lookback period

### 4. Output Table + CSV

In [ ]:
if EXPORT:
    vol_df.to_csv("fx_volatility_regimes_output.csv")
    disp_df.to_csv("fx_dispersion_regimes_output.csv")

### 5. Plot Volatility

In [ ]:
plot_fx(vol_df, 'Volatility', 'Vol')

### 6. Plot Dispersion

In [ ]:
regime_counts = (
    disp_df.groupby(disp_df.index.to_period("Y"))["Regime"]
    .value_counts()
    .unstack()
    .fillna(0)
    .astype(int)
)

regime_counts.index = regime_counts.index.astype(str)  # ensure x-axis is label-friendly

regime_counts.plot(kind="bar", stacked=True, figsize=(12, 4))
plt.title("Dispersion Regime Counts by Year")
plt.xlabel("Year")
plt.ylabel("Day Count")
plt.tight_layout()
if EXPORT:
    plt.savefig("dispersion_regime_barplot.png")
plt.show()